# Import libraries

In [7]:
#!/usr/bin/python
import sqlite3 as sq3
from datetime import datetime
import requests
import json
import socket
from datetime import time
import logging


# Print_log function

In [8]:
logging.basicConfig(filename = '%s.log'%str(datetime.now().strftime("%Y_%m_%d_%H_%M_%S")), level = logging.DEBUG, format='[%(filename)s:%(lineno)s - %(funcName)20s() ] %(asctime)s %(message)s')

def print_log(message = None, value = None): # Function to logging and printing messages into terminal for debug
    logging.info(message)
    if value != None:
        logging.info(value)
    print(message)
    if value != None:
        print(value)
    return 0

# Create databases and tables

In [9]:
# First table draft
cur = sq3.connect("main_database.db")
print("Opened database successfully")

# Here maybe required a zero table 

# //Maybe required zero table data added time. DATA_ADDED_TIME

cur.execute('''CREATE TABLE ZERO
            (ID              INTEGER PRIMARY KEY    NOT NULL,
            ANIMAL_ID        TEXT                   UNIQUE,
            ADDED_TIME       DATETIME);''')

print("ZERO table created successfully")
cur.close()


cur = sq3.connect("main_database.db")
print("Opened database successfully")

cur.execute('''CREATE TABLE EQUIPMENT
            (EQUIPMENT_ID               INTEGER PRIMARY KEY  AUTOINCREMENT,
            TYPE                        TEXT CHECK( TYPE IN ('SCALES','SCALES_SPRAY','FEEDER_SCALES') )  DEFAULT 'NULL',
            MODEL                       TEXT,
            EQUIPMENT_NAME              TEXT    UNIQUE,
            LOCATION                    TEXT,
            PERSON                      TEXT,
            CONTACT                     TEXT);''')

print("EQUIPMENT table created successfully")
cur.close()


cur = sq3.connect("main_database.db")
print("Opened database successfully")

cur.execute('''CREATE TABLE MAIN_DATA
            (CASE_ID                    INTEGER PRIMARY KEY  AUTOINCREMENT,
            ANIMAL_ID                   TEXT                NOT NULL,
            EVENT_TIME                  DATETIME            NOT NULL,
            WEIGHT                      DOUBLE              NOT NULL,
            EQUIPMENT_NAME              TEXT                NOT NULL,
            DRINK_DURATION              TIME,               
            DATA_STATUS                 TEXT CHECK( DATA_STATUS IN ('YES','NO') )   NOT NULL DEFAULT 'NO',
            TRANSFER_TIME               DATETIME,
            FOREIGN KEY(ANIMAL_ID) REFERENCES ZERO(ANIMAL_ID),
            FOREIGN KEY(EQUIPMENT_NAME) REFERENCES EQUIPMENT(EQUIPMENT_NAME));''')


print("MAIN_DATA table created successfully")
cur.close()


cur = sq3.connect("main_database.db")
print("Opened database successfully")

cur.execute('''CREATE TABLE RAW_DATA
            (RAW_CASE_ID                INTEGER PRIMARY KEY      AUTOINCREMENT,  
            ANIMAL_ID                   TEXT                 NOT NULL,
            EVENT_TIME                  DATETIME             NOT NULL,
            WEIGHT                      DOUBLE               NOT NULL,
            EQUIPMENT_NAME              TEXT                 NOT NULL,
            DATA_STATUS                 TEXT CHECK( DATA_STATUS IN ('YES','NO') )   NOT NULL DEFAULT 'NO', 
            TRANSFER_TIME               DATETIME,
            FOREIGN KEY(ANIMAL_ID) REFERENCES ZERO(ANIMAL_ID),
            FOREIGN KEY(EQUIPMENT_NAME) REFERENCES EQUIPMENT(EQUIPMENT_NAME));''')

print("RAW_DATA table Created successfully")
cur.close()


cur = sq3.connect("main_database.db")
print("Opened database successfully")

# //COMMAND_TIME change to ORDER_TIME
# //Change order between COMMAND_TIME and SPRAY_STATUS

cur.execute('''CREATE TABLE SPRAY
            (CASE_ID                        INTEGER PRIMARY KEY  AUTOINCREMENT,
            ANIMAL_ID                       TEXT                    NOT NULL,
            EQUIPMENT_NAME                  TEXT                    NOT NULL,
            TYPE                            TEXT CHECK( TYPE IN ('PAINT','DRUG') )   NOT NULL DEFAULT 'DRUG',                  
            ORDER_TIME                      DATETIME,
            SPRAY_STATUS                    TEXT CHECK( SPRAY_STATUS IN ('NONE','DONE', 'WAIT') )   NOT NULL DEFAULT 'NONE',
            DONE_TIME                       DATETIME,
            DATA_STATUS                     TEXT CHECK( DATA_STATUS IN ('YES','NO') )   NOT NULL DEFAULT 'NO',
            TRANSFER_TIME                   DATETIME,
            FOREIGN KEY(ANIMAL_ID) REFERENCES ZERO(ANIMAL_ID),
            FOREIGN KEY(EQUIPMENT_NAME) REFERENCES EQUIPMENT(EQUIPMENT_NAME));''')

print("SPRAY table created successfully")

cur = sq3.connect("main_database.db")
print("Opened database successfully")

cur.execute('''CREATE TABLE METRICS
            (METRICS_ID                     INTEGER PRIMARY KEY  AUTOINCREMENT,
            EQUIPMENT_NAME                  INTEGER                    NOT NULL,
            BOOT_ID                         INTEGER                    NOT NULL,
            EVENT_TIME                      DATETIME,                   
            UPTIME                          DATETIME,
            LAST_SHUTDOWN                   DATETIME,
            CPU_TEMP                        DOUBLE,
            VOLTAGE                         DOUBLE,
            ERROR                           TEXT,
            MESSAGE                         TEXT,
            MEMORY                          DOUBLE,
            FOREIGN KEY(EQUIPMENT_NAME) REFERENCES EQUIPMENT(EQUIPMENT_NAME));''')

print("METRICS table created successfully")



cur.close()

Opened database successfully
ZERO table created successfully
Opened database successfully
EQUIPMENT table created successfully
Opened database successfully
MAIN_DATA table created successfully
Opened database successfully
RAW_DATA table Created successfully
Opened database successfully
SPRAY table created successfully
Opened database successfully
METRICS table created successfully


# Staging animal into SPRAY table with wait status

In [24]:
############################################################################
# Staging animals into spray from zero table
# weakness #1 of this code is if cow is in other database maybe it will be skipped
# weakness #2 if cow not came in this scales maybe it will be skipped

def staging_into_spray_table():
    try:
        # function variables 
        spray_type = "DRUG"
        spray_status = "WAIT"
        order_time = datetime.now()

        cur = sq3.connect('main_database.db')

        ##################
        # Get equipment name from equipment table
        cursor_equipment = cur.execute("SELECT EQUIPMENT_NAME from EQUIPMENT")
        for row in cursor_equipment:
            equipment_name = row[0]
            print_log("EQUIPMENT_NAME = ", equipment_name)
        ###################

        ###################
        # get unique animal id from zero table and staging into spray status
        cursor_zero = cur.execute("SELECT ID, ANIMAL_ID from ZERO")
        for row in cursor_zero:
            id = row[0]
            print_log("ID = ", id)
            animal_id = row[1]
            print_log("ANIMAL_ID = ", animal_id)
            cur.execute("INSERT INTO SPRAY (ANIMAL_ID, EQUIPMENT_NAME, TYPE, SPRAY_STATUS, ORDER_TIME) VALUES (?, ?, ?, ?, ?)",
                            (animal_id, equipment_name, spray_type, spray_status, order_time))
        cur.commit()
        cur.close()
    except Exception as e:
        print_log("Error in adding data into SPRAY table", e)
    else:
        print_log("Success: Data into SPRAY table added")
        return 0

############################################################################

In [25]:
staging_into_spray_table()

EQUIPMENT_NAME = 
Test_sqlite_ver45
ID = 
1
ANIMAL_ID = 
02821760656c
ID = 
2
ANIMAL_ID = 
00161710651f
Success: Data into SPRAY table added


0

# Spray function by animal id

In [27]:
#####################################################################
# function of spray command by check of animal_id and spray_status
def spray_animal_by_spray_status(animal_id):
    try:
        print_log("Start spray function")
        cur = sq3.connect('main_database.db')
        cursor_spray_animal_id = cur.execute("SELECT CASE_ID, ANIMAL_ID, SPRAY_STATUS from SPRAY")
        for row in cursor_spray_animal_id:
            case_id = row[0]
            print_log("CASE_ID = ", case_id)
            spray_animal_id = row[1]
            print_log("SPRAY_ANIMAL_ID = ", spray_animal_id)
            spray_status = row[2]
            print_log("SPRAY_STATUS = ", spray_status)

            if spray_animal_id == animal_id:
                if spray_status == "WAIT":
                    # RUN GPIO PWM function
                    data_for_query = ('DONE', datetime.now() , case_id)
                    sqlite_querry = """UPDATE SPRAY SET SPRAY_STATUS = ?, DONE_TIME = ? WHERE CASE_ID = ?"""                 
                    cur.execute(sqlite_querry, data_for_query) 

        cur.commit()
        cur.close()
    except Exception as e:
        print_log("Error in Spray function", e)
    else:
        print_log("Success: Animal sprayed")
        return 0
#####################################################################

In [30]:
spray_animal_by_spray_status('00161710651f')

Start spray function
CASE_ID = 
1
SPRAY_ANIMAL_ID = 
02821760656c
SPRAY_STATUS = 
DONE
CASE_ID = 
2
SPRAY_ANIMAL_ID = 
00161710651f
SPRAY_STATUS = 
WAIT
Success: Animal sprayed


0

# Insert test data

In [57]:
cur = sq3.connect('main_database.db')
print("Opened database successfully")

cur.execute("INSERT INTO FIRST (ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID, DATA_STATUS, TRANSFER_TIME) \
    VALUES ('rfid_sql_test_1', datetime(strftime('%s','now'), 'unixepoch', 'localtime'), 300, 'model_sql_test', 'NO', NULL)");

cur.execute("INSERT INTO FIRST (ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID, DATA_STATUS, TRANSFER_TIME) \
    VALUES ('rfid_sql_test_2', datetime(strftime('%s','now'), 'unixepoch', 'localtime'), 300, 'model_sql_test', 'NO', NULL)");

cur.execute("INSERT INTO FIRST (ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID, DATA_STATUS, TRANSFER_TIME) \
    VALUES ('rfid_sql_test_3', datetime(strftime('%s','now'), 'unixepoch', 'localtime'), 300, 'model_sql_test', 'NO', NULL)");

cur.execute("INSERT INTO RAW_DATA(ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID) \
    VALUES ('rfid_sql_test', datetime(strftime('%s','now'), 'unixepoch', 'localtime'), 290, 'model_sql_test')");

cur.execute("INSERT INTO RAW_DATA(ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID) \
    VALUES ('rfid_sql_test', datetime(strftime('%s','now'), 'unixepoch', 'localtime'), 300, 'model_sql_test')");

cur.execute("INSERT INTO RAW_DATA(ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID) \
    VALUES ('rfid_sql_test', datetime(strftime('%s','now'), 'unixepoch', 'localtime'), 300, 'model_sql_test')");

cur.execute("INSERT INTO RAW_DATA(ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID) \
    VALUES ('rfid_sql_test', datetime(strftime('%s','now'), 'unixepoch', 'localtime'), 300, 'model_sql_test')");

cur.execute("INSERT INTO RAW_DATA(ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID) \
    VALUES ('rfid_sql_test', datetime(strftime('%s','now'), 'unixepoch', 'localtime'), 300, 'model_sql_test')");


cur.commit()
print("Recodrs created succesfully")
cur.close()

Opened database successfully
Recodrs created succesfully


In [ ]:
# print MAIN_DATA_TABLE cells
conn = sq3.connect('main_database.db')
print("Opened database successfully")

cursor = conn.execute("SELECT CASE_ID, ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID, DRINK_DURATION, DATA_STATUS, TRANSFER_TIME from FIRST")

for row in cursor:
    print("ID = ", row[0])
    print("ANIMAL_ID = ", row[1])
    print("EVENT_TIME = ", row[2])
    print("WEIGHT = ", row[3])
    print("SCALES_TYPE = ", row[4])
    print("LAST_DRINK_DURATION = ", row[5])
    print("DATA_STATUS = ", row[6])
    print("DATA_TRANSFER_TIME = ", row[7], "\n")

print("Operation done succesfully")
conn.close()


# Merge info in different databases

In [ ]:
conn_1 = sq3.connect('main_database_1.db')
conn_2 = sq3.connect('main_database_2.db')
print("Opened database successfully")









# Insert new animal_id in zero table

In [14]:
###################################################################################################
# Insert to zero table new unique animal_id 
def insert_new_unique_animal_id(animal_id):
    try:
        conn = sq3.connect('main_database.db')
        print_log("Opened database successfully")
        cursor = conn.execute("SELECT ANIMAL_ID from ZERO")           
        print_log("animal_id", animal_id)
        print_log("Start to add new unique animal id")
        data_for_query = (animal_id)
        conn.execute("INSERT INTO ZERO (ANIMAL_ID) VALUES (?)",
                    (animal_id,))

        print_log("animal_id", animal_id)
        conn.commit()
        conn.close()

    except Exception as e:
        print_log("Error in creating new animal_id in zero table ", e)
    else:
        return 0
###################################################################################################


In [17]:
insert_new_unique_animal_id("12")

Opened database successfully
animal_id
12
Start to add new unique animal id
animal_id
12


0

# Insert new EQUIPMENT_ID in EQUIPMENT table

In [41]:
###################################################################################################
# Insert to zero table new unique equipment data
def Insert_New_Unique_Equipment_Type_Model(type, model, equipment_name, location, person, contact):
    try:
        conn = sq3.connect('main_database.db')
        print_log("Opened database successfully")
        cursor = conn.execute("SELECT TYPE, MODEL, EQUIPMENT_NAME, LOCATION, PERSON, CONTACT from EQUIPMENT")           
        print_log("Start to add new unique equipment data")
        data_for_query = (type, model, equipment_name, location, person, contact)
        conn.execute("INSERT INTO EQUIPMENT (TYPE, MODEL, EQUIPMENT_NAME, LOCATION, PERSON, CONTACT) VALUES (?, ?, ?, ?, ?, ?)",
                    (type, model, equipment_name, location, person, contact))

        print_log("TYPE", type)
        print_log("MODEL", model)
        print_log("EQUIPMENT_NAME", equipment_name)
        print_log("LOCATION", location)
        print_log("PERSON", person)
        print_log("CONTACT", contact)
        conn.commit()
        conn.close()

    except Exception as e:
        print_log("Error in creating new unique equipment data in EQUIPMENT table ", e)
    else:
        return 0
###################################################################################################

In [42]:
Insert_New_Unique_Equipment_Type_Model("SCALES_SPRAY", "800", "PCF_MODEL_10", "Agrarka_1212", "Yerkebulan", "+77779980051")

Opened database successfully
Start to add new unique equipment data
TYPE
SCALES_SPRAY
MODEL
800
EQUIPMENT_NAME
PCF_MODEL_10
LOCATION
Agrarka_1212
PERSON
Yerkebulan
CONTACT
+77779980051


0

# Insert into MAIN_DATA_TABLE

In [39]:
# Collect to database function 
##########################################################################################

def collect_to_first_table(animal_id, weight, equipment_id):
    try:
        data_status = 'NO'
        drink_duration= 'NULL'
        event_time = datetime.now()
        transfer_time = 'NULL'

        conn = sq3.connect('main_database.db')
        print("Opened database successfully")
        conn.execute("INSERT INTO FIRST (ANIMAL_ID, EVENT_TIME, WEIGHT, EQUPMENT_ID, DRINK_DURATION, DATA_STATUS, TRANSFER_TIME ) VALUES(?, ?, ?, ?, ?, ?, ?)",
                                            (animal_id, event_time, weight, equipment_id, drink_duration, data_status, transfer_time))
        conn.commit()
        conn.close()

    except Exception as e:
        print("Error to save data in ", e)
    else:
        return 0
###########################################################################

In [83]:
# main table collect check
animal_id = 'test_main_data_table_function'
weight = 1231
equipment_id = 'sqlite_test'

collect_to_first_table(animal_id, weight, equipment_id)

Opened database successfully


0

# Insert into RAW DATA TABLE

In [51]:
###################################################################################################3
def collect_to_raw_data_table(animal_id, weight, equipment_id):
    try:
        data_status = 'NO'
        event_time = datetime.now()
        transfer_time = 'NULL'

        conn = sq3.connect('main_database.db')
        print("Opened database successfully")
        conn.execute("INSERT INTO RAW_DATA (ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID, RADATA_STATUS, TRANSFER_TIME ) VALUES(?, ?, ?, ?, ?, ?)",
                                            (animal_id, event_time, weight, equipment_id, data_status, transfer_time))
        conn.commit()
        conn.close()

    except Exception as e:
        print("Error to save data in ", e)
    else:
        return 0
##################################################################################################

In [55]:
# raw table collect check
animal_id = 'raw_test_data_table_function'
weight = 500
scales_type = 'sqlite_test'

collect_to_raw_data_table(animal_id, weight, scales_type)

Opened database successfully


0

# Insert spray data into table

In [ ]:
cur = sq3.connect('main_database.db')
print("Opened database successfully")

cur.execute("INSERT INTO SPRAY (CASE_ID, ANIMAL_ID, EQUPMWENT_ID, TYPE, STATUS, WAIT_TIME, DONE_TIME, DATA_STATUS, TRANSFER_TIME) \
    VALUES ('rfid_sql_test', datetime(strftime('%s','now'), 'unixepoch', 'localtime'), 300, 'model_sql_test', 'DONE', NULL, 'NO')");

cur.commit()
print("Recodrs created succesfully")
cur.close()

# Check spray status and insert into yes status

In [ ]:
def spray(animal_id):
    try:
        conn = sq3.connect('main_database.db')
        print("Opened database successfully")

        cursor = conn.execute("SELECT CASE_ID, ANIMAL_ID, EQUPMWENT_ID, TYPE, STATUS, WAIT_TIME, DONE_TIME, DATA_STATUS, TRANSFER_TIME from SPRAY")

        for row in cursor:
            if row[5] == 'WAIT': # check status in table

                #print("CASE_ID = ", row[0])
                case_id = row[0]
                #print("EVENT_TIME = ", row[1])
                animal_id = row[1]
                #print("EVENT_TIME = ", row[2])
                event_time = row[2]
                #print("WEIGHT = ", row[3])
                weight = row[3]
                #print("SCALES_TYPE = ", row[4])
                scales_type = row[4]
                #print("DATA_STATUS = ", row[7])
                data_status = row[6]
                #case_id = 

                # Function of sending data from database to smart-farm server
                # def json_send_packet()
                
                #print("START SEND DATA TO SERVER:")
                url = 'http://194.4.56.86:8501/api/weights'
                headers = {'Content-type': 'application/json'}
                data = {"AnimalNumber" : animal_id,
                        "Date" : event_time,
                        "Weight" : weight,
                        "ScalesModel" : scales_type}
                answer = requests.post(url, data=json.dumps(data), headers=headers, timeout=10)
                print("Answer from server: ", answer) # Is it possible to stop on this line in the debug?
                print(answer.content)
                print(row[0])

                # Change status of DATA_STATUS in cows table from main_database 
                if 200 == answer.status_code:
                    #print("This is part of change status in cows table Data status")
                    # change status
                    data_for_query = ('YES', datetime.now() , row[0])
                    sqlite_querry = """UPDATE FIRST SET DATA_STATUS = ?, TRANSFER = ? WHERE CASE_ID = ?"""                 
                    conn.execute(sqlite_querry, data_for_query)
                    conn.commit()
    except Exception as e:

    else:
    

# Spray All function

In [ ]:
# Spray all function # set al cows to spray



# Send to server draft function

In [ ]:
###################################################################################################3
def Send_data_to_server(): # Sending data into Igor's server through JSON
    try:
        print("Extract data from database")

        # Extract info from cows table of main_database
        #    
        conn = sq3.connect('main_database.db')
        print("Opened database successfully")

        cursor = conn.execute("SELECT CASE_ID, ANIMAL_ID, EVENT, WEIGHT, EQUIPMENT, DRINK_DURATION, DATA_STATUS, TRANSFER from FIRST")

        for row in cursor:
            if row[6] == 'NO': # check status in table
                #print("ANIMAL_ID = ", row[1])
                animal_id = row[1]
                #print("EVENT_TIME = ", row[2])
                event_time = row[2]
                #print("WEIGHT = ", row[3])
                weight = row[3]
                #print("SCALES_TYPE = ", row[4])
                scales_type = row[4]
                #print("DATA_STATUS = ", row[7])
                data_status = row[6]
                #case_id = 

                # Function of sending data from database to smart-farm server
                # def json_send_packet()
                
                #print("START SEND DATA TO SERVER:")
                url = 'http://194.4.56.86:8501/api/weights'
                headers = {'Content-type': 'application/json'}
                data = {"AnimalNumber" : animal_id,
                        "Date" : event_time,
                        "Weight" : weight,
                        "ScalesModel" : scales_type}
                answer = requests.post(url, data=json.dumps(data), headers=headers, timeout=10)
                print("Answer from server: ", answer) # Is it possible to stop on this line in the debug?
                print(answer.content)
                print(row[0])

                # Change status of DATA_STATUS in cows table from main_database 
                if 200 == answer.status_code:
                    #print("This is part of change status in cows table Data status")
                    # change status
                    data_for_query = ('YES', datetime.now() , row[0])
                    sqlite_querry = """UPDATE FIRST SET DATA_STATUS = ?, TRANSFER = ? WHERE CASE_ID = ?"""                 
                    conn.execute(sqlite_querry, data_for_query)
                    conn.commit()
          
                    
    except Exception as e:
        print("Error send data to server", e)
    else:
        print("Operation update database and sending to server done succesfully")
        conn.close()
        return 0

#######################################################################################

# internet connection check function
def check_internet_connection():
    """ Returns True if there's a connection """

    IP_ADDRESS_LIST = [
        "1.1.1.1",  # Cloudflare
        "1.0.0.1",
        "8.8.8.8",  # Google DNS
        "8.8.4.4",
        "208.67.222.222",  # Open DNS
        "208.67.220.220"
    ]

    port = 53
    timeout = 3

    for host in IP_ADDRESS_LIST:
        try:
            socket.setdefaulttimeout(timeout)
            socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect((host, port))
            return True
        except socket.error:
            pass
    else:
        print("No internet connection")
        return False 

# Check internet connection 
if check_internet_connection() == True :
    Send_data_to_server()
    Send_raw_data_to_server()
else:
    time.sleep(10)

# Send to server from MAIN TABLE DATA

In [80]:
# This function is select data from cows database then send is to Igos server
########################################################################################
def Send_data_to_server(): # Sending data into Igor's server through JSON
    try:
        print("Extract data from database")

        # Extract info from cows table of main_database
        #    
        conn = sq3.connect('main_database.db')
        print("Opened database successfully")

        cursor = conn.execute("SELECT CASE_ID, ANIMAL_ID, EVENT_TIME, WEIGHT, SCALES_TYPE, LAST_DRINK_DURATION, MAIN_DATA_STATUS, TRANSFER_TIME from MAIN_DATA_TABLE")

        for row in cursor:
            if row[6] == 'NO': # check status in table
                #print("ANIMAL_ID = ", row[1])
                animal_id = row[1]
                #print("EVENT_TIME = ", row[2])
                event_time = row[2]
                #print("WEIGHT = ", row[3])
                weight = row[3]
                #print("SCALES_TYPE = ", row[4])
                scales_type = row[4]
                #print("DATA_STATUS = ", row[7])
                data_status = row[6]
                #case_id = 

                # Function of sending data from database to smart-farm server
                # def json_send_packet()
                
                #print("START SEND DATA TO SERVER:")
                url = 'http://194.4.56.86:8501/api/weights'
                headers = {'Content-type': 'application/json'}
                data = {"AnimalNumber" : animal_id,
                        "Date" : event_time,
                        "Weight" : weight,
                        "ScalesModel" : scales_type}
                answer = requests.post(url, data=json.dumps(data), headers=headers, timeout=10)
                print("Answer from server: ", answer) # Is it possible to stop on this line in the debug?
                print(answer.content)
                print(row[0])

                # Change status of DATA_STATUS in cows table from main_database 
                if 200 == answer.status_code:
                    #print("This is part of change status in cows table Data status")
                    # change status
                    data_for_query = ('YES', datetime.now() , row[0])
                    sqlite_querry = """UPDATE MAIN_DATA_TABLE SET MAIN_DATA_STATUS = ?, DATA_TRANSFER_TIME = ? WHERE CASE_ID = ?"""                 
                    conn.execute(sqlite_querry, data_for_query)
                    conn.commit()
          
                    
    except Exception as e:
        print("Error send data to server", e)
    else:
        print("Operation update database and sending to server done succesfully")
        conn.close()
        return 0

###########################################################################################

Send_data_to_server()

Extract data from database
Opened database successfully
Answer from server:  <Response [200]>
b'101332'
1
Answer from server:  <Response [200]>
b'101333'
2
Answer from server:  <Response [200]>
b'101334'
3
Answer from server:  <Response [200]>
b'101335'
4
Answer from server:  <Response [200]>
b'101336'
5
Operation update database and sending to server done succesfully


0

# Send to server from RAW TABLE DATA

In [ ]:
# This function is select data from raw database then send is to Igos server
###########################################################################################################
def Send_raw_data_to_server(): # Sending data into Igor's server through JSON
    try:
        print("Extract data from raw database")

        # Extract info from cows table of main_database
        #    
        conn = sq3.connect('main_database.db')
        print("Opened raw database successfully")

        cursor = conn.execute("SELECT CASE_ID, ANIMAL_ID, EVENT_TIME, WEIGHT, EQUIPMENT_ID, DATA_STATUS, TRANSFER_TIME from RAW_DATA")

        for row in cursor:
            if row[6] == 'NO': # check status in table
                #print("ANIMAL_ID = ", row[1])
                animal_id = row[1]
                #print("EVENT_TIME = ", row[2])
                event_time = row[2]
                #print("WEIGHT = ", row[3])
                weight = row[3]
                #print("EQUIPMENT_ID = ", row[4])
                equipment_id = row[4]
                #print("DATA_STATUS = ", row[5])
                data_status = row[5]
                #case_id = 

                # Function of sending data from database to smart-farm server
                # def json_send_packet()
                
                #print("START SEND DATA TO SERVER:")
                url = 'http://194.4.56.86:8501/api/RawWeights'
                headers = {'Content-type': 'application/json'}
                data = {"AnimalNumber" : animal_id,
                        "Date" : event_time,
                        "Weight" : weight,
                        "ScalesModel" : equipment_id}
                answer = requests.post(url, data=json.dumps(data), headers=headers, timeout=10)
                print("Answer from server: ", answer) # Is it possible to stop on this line in the debug?
                print(answer.content)
                print(row[0])

                # Change status of DATA_STATUS in cows table from main_database 
                if 200 == answer.status_code:
                    #print("This is part of change status in cows table Data status")
                    # change status
                    data_for_query = ('YES', datetime.now() , row[0])
                    sqlite_querry = """UPDATE RAW_DATA SET DATA_STATUS = ?, TRANSFER_TIME = ? WHERE CASE_ID = ?"""                 
                    conn.execute(sqlite_querry, data_for_query)
                    conn.commit()
          
                    
    except Exception as e:
        print("Error send data to raw data server", e)
    else:
        print("Operation update raw table of database and sending to server done succesfully")
        conn.close()
        return 0
#######################################################################################


Send_raw_data_to_server()



# Internet connection test

In [66]:
# Exmple from internet
def check_internet_connection():
    """ Returns True if there's a connection """

    IP_ADDRESS_LIST = [
        "1.1.1.1",  # Cloudflare
        "1.0.0.1",
        "8.8.8.8",  # Google DNS
        "8.8.4.4",
        "208.67.222.222",  # Open DNS
        "208.67.220.220"
    ]

    port = 53
    timeout = 3

    for host in IP_ADDRESS_LIST:
        try:
            socket.setdefaulttimeout(timeout)
            socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect((host, port))
            return True
        except socket.error:
            pass
    else:
        print("No internet connection")
        return False 

print(check_internet_connection())


True


In [5]:
def print_log(message = None, value = None): # Function to logging and printing messages into terminal for debug
    #logging.info(message)
    #if value != None:
        #logging.info(value)
    #print(message)
    #if value != None:
        #print(value)
    return 0

print_log()

0

In [1]:
import logging
from datetime import datetime, date, time

In [2]:
# Test of new log file name
logging.basicConfig(filename = '%s.log'%str(datetime.now().strftime("%Y-%m-%d_%H_%M_%S")), level = logging.DEBUG, format='[%(filename)s_%(lineno)s - %(funcName)20s() ] %(asctime)s %(message)s')



def print_log(message = None, value = None): # Function to logging and printing messages into terminal for debug
    logging.info(message)
    if value != None:
        logging.info(value)
    print(message)
    if value != None:
        print(value)
    return 0


print_log('start', 'run')

start
run


0